<a href="https://colab.research.google.com/github/wldopie/ESAA/blob/main/OB_4%EC%A1%B0_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8(1)_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

아래 코드 실행하기 전에 'ESAA OB 4조' 폴더 오른쪽 클릭 후 `드라이브에 바로가기 추가`하기! 

# **1. 라이브러리 임포트 및 데이터 로딩**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/ESAA OB 4조/Project 1/Data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA OB 4조/Project 1/Data/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/ESAA OB 4조/Project 1/Data/sample_submission.csv')

---

# **2. 데이터 전처리**

## **2.1 데이터 확인**

In [ ]:
train.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [ ]:
test.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         7920 non-null   int64 
 1   송하인_격자공간고유번호  7920 non-null   int64 
 2   수하인_격자공간고유번호  7920 non-null   int64 
 3   물품_카테고리       7920 non-null   object
dtypes: int64(3), object(1)
memory usage: 247.6+ KB


  - `격자공간고유번호` 칼럼은 `수치형`, `물품_카테고리` 칼럼은 `범주형` 변수이며, 결측치는 존재하지 않는다. 

## **2.2 데이터 가공을 위한 전처리**

In [ ]:
# index 칼럼 제거
train.drop('index', axis=1, inplace=True)
test.drop('index', axis=1, inplace=True)

In [ ]:
# 격자공간고유번호는 수치형 변수보다는 범주형 변수가 적합하므로 형변환 진행
train = train.astype({'송하인_격자공간고유번호':'str', '수하인_격자공간고유번호':'str'})
test = test.astype({'송하인_격자공간고유번호':'str', '수하인_격자공간고유번호':'str'})

In [ ]:
# target 변수인 운송장_건수를 개별적으로 저장
y_train = train['운송장_건수']
train.drop('운송장_건수', axis=1, inplace=True)

In [ ]:
# train, test 데이터를 한 번에 가공하기 위해 데이터프레임 합치기
data = pd.concat([train, test], ignore_index=True)

In [ ]:
data.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,5011000595017300,2871000192069300,음반
1,4148000690043300,5011000264024400,문화컨텐츠
2,5011000078068400,1120000007005400,농산물
3,4127100048006400,5011000587019400,기타식품
4,5011000078068400,2823700010076300,농산물


In [ ]:
data.shape      # train(31684) + test(7920) = 총 39604개의 행을 가진 데이터프레임 완성

(39604, 3)

---

# **3. 데이터 가공**

## **3.1 격자공간고유번호를 활용한 파생변수 생성**
  - 송하인/수하인 `격자공간고유번호 칼럼`으로부터 `자릿수 별 파생변수 생성`
    - 격자공간고유번호 `1~2자리` : `시도`
    - 격자공간고유번호 `1~5자리` : `시군구`
    - 격자공간고유번호 `1~10자리` : `동`
    - 나머지 4자리 : 기타

In [ ]:
data['송하인_시도'] = data.iloc[:,0].str[:2]
data['송하인_시군구'] = data.iloc[:,0].str[:5]
data['송하인_동'] = data.iloc[:,0].str[:10]
data['송하인_기타'] = data.iloc[:,0].str[10:14]

data['수하인_시도'] = data.iloc[:,1].str[:2]
data['수하인_시군구'] = data.iloc[:,1].str[:5]
data['수하인_동'] = data.iloc[:,1].str[:10]
data['수하인_기타'] = data.iloc[:,1].str[10:14]

In [ ]:
# 송하인_격자공간고유번호, 수하인_격자공간고유번호는 의미없는 칼럼이 되었으므로 제거
data.drop(['송하인_격자공간고유번호','수하인_격자공간고유번호'], axis=1, inplace=True)

## **3.2 `송하인_동`, `수하인_동` 칼럼 가공**
제주도를 제외한 나머지 시도의 동은 00000으로 채우기

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39604 entries, 0 to 39603
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   물품_카테고리  39604 non-null  object
 1   송하인_시도   39604 non-null  object
 2   송하인_시군구  39604 non-null  object
 3   송하인_동    39604 non-null  object
 4   송하인_기타   39604 non-null  object
 5   수하인_시도   39604 non-null  object
 6   수하인_시군구  39604 non-null  object
 7   수하인_동    39604 non-null  object
 8   수하인_기타   39604 non-null  object
dtypes: object(9)
memory usage: 2.7+ MB


In [ ]:
data[data['송하인_시도']=='50']

,물품_카테고리,송하인_시도,송하인_시군구,송하인_동,송하인_기타,수하인_시도,수하인_시군구,수하인_동,수하인_기타
0,음반,50,50110,5011000595,0173,28,28710,2871000192,0693
2,농산물,50,50110,5011000078,0684,11,11200,1120000007,0054
4,농산물,50,50110,5011000078,0684,28,28237,2823700010,0763
5,농산물,50,50130,5013000778,0732,41,41590,4159000114,0164
6,수산,50,50110,5011000316,0243,48,48270,4827000369,0602
...,...,...,...,...,...,...,...,...,...
39597,음반,50,50110,5011000595,0173,11,11650,1165000011,0854
39598,농산물,50,50130,5013000616,0571,27,27260,2726000016,0422
39599,농산물,50,50110,5011000266,0512,46,46230,4623000417,0381
39601,농산물,50,50130,5013000610,0491,11,11470,1147000018,0914


In [ ]:
data = data.astype({'송하인_시도':'str', '수하인_시도':'str'})

In [ ]:
for i in range(len(data)):
    if data['송하인_시도'][i] != '50':
        data['송하인_동'][i] = data['송하인_시군구'][i] + '00000'

    if data['수하인_시도'][i] != '50':
        data['수하인_동'][i] = data['수하인_시군구'][i] + '00000'

In [ ]:
# 올바르게 변경되었는지 확인
data[data['송하인_시도'] != '50']

,물품_카테고리,송하인_시도,송하인_시군구,송하인_동,송하인_기타,수하인_시도,수하인_시군구,수하인_동,수하인_기타
1,문화컨텐츠,41,41480,4148000000,0433,50,50110,5011000264,0244
3,기타식품,41,41271,4127100000,0064,50,50110,5011000587,0194
8,아우터,41,41360,4136000000,0941,50,50130,5013000823,0534
12,신발,41,41463,4146300000,0473,50,50110,5011000263,0691
16,기타식품,47,47250,4725000000,0214,50,50110,5011000596,0122
...,...,...,...,...,...,...,...,...,...
39587,주방용품,41,41410,4141000000,0301,50,50110,5011000376,0131
39593,가공식품,41,41360,4136000000,0803,50,50110,5011000199,0022
39596,기타식품,47,47113,4711300000,0334,50,50110,5011000215,0652
39600,문화컨텐츠,11,11545,1154500000,0983,50,50110,5011000264,0551


In [ ]:
data['송하인_동'].value_counts()

5011000078    4876
5013000610    2296
5013000731    1751
4148000000    1494
5011000137    1427
              ... 
5011000811       1
5013000010       1
5013000091       1
5011000231       1
5013000852       1
Name: 송하인_동, Length: 813, dtype: int64

## **3.3 라벨 인코딩**

In [ ]:
from sklearn.preprocessing import LabelEncoder

### 3.3.1 물품_카테고리

In [ ]:
encoder1 = LabelEncoder()

encoder1.fit(data['물품_카테고리'])

data['물품_카테고리'] = encoder1.transform(data['물품_카테고리'])

### 3.3.2 시도

In [ ]:
encoder2 = LabelEncoder()

encoder2.fit(data['송하인_시도'])

data['송하인_시도'] = encoder2.transform(data['송하인_시도'])
data['수하인_시도'] = encoder2.transform(data['수하인_시도'])

### 3.3.3 시군구

In [ ]:
encoder3 = LabelEncoder()

encoder3.fit(data['수하인_시군구'])

data['송하인_시군구'] = encoder3.transform(data['송하인_시군구'])
data['수하인_시군구'] = encoder3.transform(data['수하인_시군구'])

### 3.3.4 동

In [ ]:
data['수하인_동']

0        2871000000
1        5011000264
2        1120000000
3        5011000587
4        2823700000
            ...    
39599    4623000000
39600    5011000264
39601    1147000000
39602    3117000000
39603    5011000373
Name: 수하인_동, Length: 39604, dtype: object

In [ ]:
new = pd.concat([data['송하인_동'],data['수하인_동']])
new

0        5011000595
1        4148000000
2        5011000078
3        4127100000
4        5011000078
            ...    
39599    4623000000
39600    5011000264
39601    1147000000
39602    3117000000
39603    5011000373
Length: 79208, dtype: object

In [ ]:
encoder4 = LabelEncoder()

encoder4.fit(new)

data['송하인_동'] = encoder4.transform(data['송하인_동'])
data['수하인_동'] = encoder4.transform(data['수하인_동'])

  - Catboost의 경우 카테고리형 변수를 훈련 데이터로 입력해야 하므로, 데이터 스케일링을 진행하지 않은 데이터를 일차적으로 생성한다.

In [ ]:
# train, test 데이터 분리
cat_train = data[:len(train)]
cat_test = data[len(train):]

In [ ]:
# train 데이터와 y_train 데이터 병합
cat_train = pd.concat([cat_train, y_train], axis = 1)

In [ ]:
# 데이터 내보내기
cat_train.to_csv('cat_train.csv')
cat_test.to_csv('cat_test.csv')

## **3.4 데이터 스케일링**
  - `MinMaxScaler`를 활용해 각 데이터들을 `0과 1사이의 값으로 변환`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
scaler = MinMaxScaler()

scaler.fit(data)

new_data = scaler.transform(data)

In [ ]:
data.columns

Index(['물품_카테고리', '송하인_시도', '송하인_시군구', '송하인_동', '송하인_기타', '수하인_시도', '수하인_시군구',
       '수하인_동', '수하인_기타'],
      dtype='object')

In [ ]:
new_data = pd.DataFrame(new_data,columns=data.columns)

In [ ]:
new_data.head()

,물품_카테고리,송하인_시도,송하인_시군구,송하인_동,송하인_기타,수하인_시도,수하인_시군구,수하인_동,수하인_기타
0,0.676768,1.0,0.996,0.502632,0.163142,0.1875,0.228,0.049956,0.686808
1,0.343434,0.5,0.428,0.093860,0.424975,1.0000,0.996,0.323401,0.234642
2,0.272727,1.0,0.996,0.224561,0.677744,0.0000,0.012,0.002629,0.043303
3,0.171717,0.5,0.364,0.079825,0.053374,1.0000,0.996,0.498685,0.184290
4,0.272727,1.0,0.996,0.224561,0.677744,0.1875,0.216,0.047327,0.757301


## **3.5 train/test 데이터 분리**

In [ ]:
new_train = new_data[:len(train)]
new_test = new_data[len(train):]

In [ ]:
new_train['운송장_건수'] = y_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
new_train.to_csv('/content/drive/MyDrive/ESAA OB 4조/Project 1/Data/new_train.csv', index=False)
new_test.to_csv('/content/drive/MyDrive/ESAA OB 4조/Project 1/Data/new_test.csv', index=False)

  - 모델링을 위한 데이터 가공이 모두 완료되었다.

---

# **4. 모델링**

## **4.1 Adaboost**

### 4.1.1 train 데이터 가공

In [ ]:
# 라이브러리 임포트
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt

In [ ]:
# 1. 모든 칼럼 포함
X_train = train.drop('운송장_건수', axis=1)
y_train = train['운송장_건수']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=8)

In [ ]:
# 2. 기타 칼럼 제거
X_train_2 = train.drop(['운송장_건수','송하인_기타','수하인_기타'], axis=1)
test_2 = test.drop(['송하인_기타','수하인_기타'], axis=1)
y_train_2 = train['운송장_건수']

X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X_train_2, y_train_2, test_size=0.3, random_state=8)

In [ ]:
# 3. 동, 기타 칼럼 제거
X_train_3 = train.drop(['운송장_건수','송하인_동','수하인_동','송하인_기타','수하인_기타'], axis=1)
test_3 = test.drop(['송하인_동','수하인_동','송하인_기타','수하인_기타'], axis=1)
y_train_3 = train['운송장_건수']

X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(X_train_3, y_train_3, test_size=0.3, random_state=8)

In [ ]:
# 4. 동, 시군구, 기타 칼럼 제거
X_train_4 = train.drop(['운송장_건수','송하인_시군구','수하인_시군구','송하인_동','수하인_동','송하인_기타','수하인_기타'], axis=1)
test_4 = test.drop(['송하인_동','송하인_시군구','수하인_시군구','수하인_동','송하인_기타','수하인_기타'], axis=1)
y_train_4 = train['운송장_건수']

X_train_4, X_val_4, y_train_4, y_val_4 = train_test_split(X_train_4, y_train_4, test_size=0.3, random_state=8)

### 4.1.2 모델링

In [ ]:
# 1. 모든 칼럼을 포함한 데이터를 활용한 모델링
DTC = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(DTC)
params = {
    'base_estimator__criterion':['gini', 'entropy'],
    'base_estimator__splitter':['best','random'],
    'algorithm':['SAMME','SAMME.R'],
    'n_estimators':[100],
    'learning_rate':[0.001, 0.01, 0.1]
}

ada_grid = GridSearchCV(ada_clf, param_grid = params, scoring='accuracy', cv=3, verbose=2)
ada_grid.fit(X_train, y_train)

print(ada_grid.best_score_)
print(ada_grid.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=  16.2s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=  16.1s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=  19.8s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=  16.4s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=  15.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=  15.8s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.1, n_estimators=

In [ ]:
pred = ada_grid.predict(X_val)
accuracy_score(y_val, pred)

0.46612665684830634

In [ ]:
# 2. 기타 칼럼을 제거한 데이터를 활용한 모델링
DTC = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(DTC)
params = {
    'base_estimator__criterion':['gini', 'entropy'],
    'base_estimator__splitter':['best','random'],
    'algorithm':['SAMME','SAMME.R'],
    'n_estimators':[100],
    'learning_rate':[0.001, 0.01, 0.1]
}

ada_grid_2 = GridSearchCV(ada_clf, param_grid = params, scoring='accuracy', cv=3, verbose=2)
ada_grid_2.fit(X_train_2, y_train_2)

print(ada_grid_2.best_score_)
print(ada_grid_2.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   9.6s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   8.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   8.9s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   8.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   8.3s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   8.4s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.1, n_estimators=

In [ ]:
pred = ada_grid_2.predict(X_val_2)
accuracy_score(y_val_2, pred)

0.45381864085840523

In [ ]:
sqrt(mean_squared_error(y_val_2, pred))

7.7293667020642784

In [ ]:
# 3. 동, 기타 칼럼을 제거한 데이터를 활용한 모델링
DTC = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(DTC)
params = {
    'base_estimator__criterion':['gini', 'entropy'],
    'base_estimator__splitter':['best','random'],
    'algorithm':['SAMME','SAMME.R'],
    'n_estimators':[100],
    'learning_rate':[0.001, 0.01, 0.1]
}

ada_grid_3 = GridSearchCV(ada_clf, param_grid = params, scoring='accuracy', cv=3, verbose=2)
ada_grid_3.fit(X_train_3, y_train_3)

print(ada_grid_3.best_score_)
print(ada_grid_3.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   3.8s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   3.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   4.2s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   4.4s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   4.2s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   4.4s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.1, n_estimators=

In [ ]:
pred_ada_3 = ada_grid_3.predict(X_val_3)
print(accuracy_score(y_val_3, pred_ada_3))
sqrt(mean_squared_error(y_val_3, pred_ada_3))

0.5239848516726278


7.3940727764783745

In [ ]:
# 4. 동, 시군구, 기타 칼럼을 제거한 데이터를 활용한 모델링
DTC = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(DTC)
params = {
    'base_estimator__criterion':['gini', 'entropy'],
    'base_estimator__splitter':['best','random'],
    'algorithm':['SAMME','SAMME.R'],
    'n_estimators':[100],
    'learning_rate':[0.001, 0.01, 0.1]
}

ada_grid_4 = GridSearchCV(ada_clf, param_grid = params, scoring='accuracy', cv=3, verbose=2)
ada_grid_4.fit(X_train_4, y_train_4)

print(ada_grid_4.best_score_)
print(ada_grid_4.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   1.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   1.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.001, n_estimators=100; total time=   1.6s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   1.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   1.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.01, n_estimators=100; total time=   1.7s
[CV] END algorithm=SAMME, base_estimator__criterion=gini, base_estimator__splitter=best, learning_rate=0.1, n_estimators=

In [ ]:
pred_ada_4 = ada_grid_4.predict(X_val_4)
print(accuracy_score(y_val_4, pred_ada_4))
sqrt(mean_squared_error(y_val_4, pred_ada_4))

0.5499684409846413


7.423492939491518

## **4.2 DecisionTree**

In [ ]:
# 라이브러리 임포트
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# 1. 모든 칼럼을 포함한 데이터를 활용한 모델링
dt_reg = DecisionTreeRegressor()

params = {
    'max_depth' : [2, 6, 10, 16, 20],
    'min_samples_leaf' : [2, 6, 10],
    'min_samples_split' :[2, 6, 10],
    'max_features' : [3,5,7],
    'max_leaf_nodes' : [2, 6, 10]
}

dt_grid = GridSearchCV(dt_reg, param_grid=params, cv=3)
dt_grid.fit(X_train, y_train)
print(dt_grid.best_score_)
print(dt_grid.best_params_)

NameError: ignored

In [ ]:
pred_dt = dt_grid.predict(X_val)
sqrt(mean_squared_error(y_val, pred_dt))

7.080668977883946

In [ ]:
# 2. 기타 칼럼 제거한 데이터를 활용한 모델링
dt_reg = DecisionTreeRegressor()

params = {
    'max_depth' : [2, 6, 10, 16, 20],
    'min_samples_leaf' : [2, 6, 10],
    'min_samples_split' :[2, 6, 10],
    'max_features' : [3,5,7],
    'max_leaf_nodes' : [2, 6, 10]
}

dt_grid_2 = GridSearchCV(dt_reg, param_grid=params, cv=3, verbose=1)
dt_grid_2.fit(X_train_2, y_train_2)
print(dt_grid_2.best_score_)
print(dt_grid_2.best_params_)

Fitting 3 folds for each of 405 candidates, totalling 1215 fits
0.0906672372065674
{'max_depth': 6, 'max_features': 5, 'max_leaf_nodes': 10, 'min_samples_leaf': 2, 'min_samples_split': 6}


In [ ]:
pred_dt_2 = dt_grid_2.predict(X_val_2)
sqrt(mean_squared_error(y_val_2, pred_dt_2))

6.554060156130839

In [ ]:
# 3. 동, 기타 칼럼 제거한 데이터를 활용한 모델링
dt_reg = DecisionTreeRegressor()

params = {
    'max_depth' : [2, 6, 10, 16, 20],
    'min_samples_leaf' : [2, 6, 10],
    'min_samples_split' :[2, 6, 10],
    'max_features' : [3,5,7],
    'max_leaf_nodes' : [2, 6, 10]
}

dt_grid_3 = GridSearchCV(dt_reg, param_grid=params, cv=3)
dt_grid_3.fit(X_train_3, y_train_3)
print(dt_grid_3.best_score_)
print(dt_grid_3.best_params_)

0.12047501267666205
{'max_depth': 20, 'max_features': 3, 'max_leaf_nodes': 6, 'min_samples_leaf': 2, 'min_samples_split': 6}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 308, in fit
    raise ValueError("max_features must be in (0, n_features]")
ValueError: max_features must be in 

In [ ]:
pred_dt_3 = dt_grid_3.predict(X_val_3)
sqrt(mean_squared_error(y_val_3, pred_dt_3))

6.838345929723773

In [ ]:
# 4. 동, 시군구, 기타 칼럼 제거한 데이터를 활용한 모델링
dt_reg = DecisionTreeRegressor()

params = {
    'max_depth' : [2, 6, 10, 16, 20],
    'min_samples_leaf' : [2, 6, 10],
    'min_samples_split' :[2, 6, 10],
    'max_features' : [3,5,7],
    'max_leaf_nodes' : [2, 6, 10]
}

dt_grid_4 = GridSearchCV(dt_reg, param_grid=params, cv=3)
dt_grid_4.fit(X_train_4, y_train_4)
print(dt_grid_4.best_score_)
print(dt_grid_4.best_params_)

0.03076122762393378
{'max_depth': 6, 'max_features': 3, 'max_leaf_nodes': 10, 'min_samples_leaf': 6, 'min_samples_split': 2}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
810 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
810 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 308, in fit
    raise ValueError("max_features must be in (0, n_features]")
ValueError: max_features must be in 

In [ ]:
pred_dt_4 = dt_grid_4.predict(X_val_4)
sqrt(mean_squared_error(y_val_4, pred_dt_4))

6.9512462382404205

## **4.3 Random Forest**

In [ ]:
# 라이브러리 임포트
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 1. 모든 칼럼을 포함한 데이터를 활용한 모델링
rf_reg = RandomForestRegressor()
params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_grid = GridSearchCV(rf_reg, param_grid = params, scoring='neg_mean_absolute_error',cv=3)
rf_grid.fit(X_train, y_train)

print(rf_grid.best_score_)
print(rf_grid.best_params_)

-2.320164178114574
{'max_depth': 8, 'min_samples_leaf': 12, 'min_samples_split': 20, 'n_estimators': 200}


In [ ]:
pred_rf = rf_grid.predict(X_val)
sqrt(mean_squared_error(y_val, pred_rf))

6.773223842128496

In [ ]:
# 2. 기타 칼럼 제거한 데이터를 활용한 모델링
rf_reg = RandomForestRegressor()
params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_grid_2 = GridSearchCV(rf_reg, param_grid = params,scoring='neg_mean_absolute_error', cv=3)
rf_grid_2.fit(X_train_2, y_train_2)

print(rf_grid_2.best_score_)
print(rf_grid_2.best_params_)

-2.321128617750775
{'max_depth': 8, 'min_samples_leaf': 18, 'min_samples_split': 16, 'n_estimators': 50}


In [ ]:
pred_rf_2 = rf_grid_2.predict(X_val_2)
sqrt(mean_squared_error(y_val_2, pred_rf_2))

6.799023816635156

In [ ]:
# 3. 동, 기타 칼럼 제거한 데이터를 활용한 모델링
rf_reg = RandomForestRegressor()
params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_grid_3 = GridSearchCV(rf_reg, param_grid = params, scoring='neg_mean_absolute_error',cv=3)
rf_grid_3.fit(X_train_3, y_train_3)

print(rf_grid_3.best_score_)
print(rf_grid_3.best_params_)

-2.31976310380799
{'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 16, 'n_estimators': 200}


In [ ]:
pred_rf_3 = rf_grid_3.predict(X_val_3)
sqrt(mean_squared_error(y_val_3, pred_rf_3))

6.590564165886113

In [ ]:
# 4. 동, 기타, 시군구 칼럼 제거한 데이터를 활용한 모델링
rf_reg = RandomForestRegressor()
params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_grid_4 = GridSearchCV(rf_reg, param_grid = params,scoring='neg_mean_absolute_error', cv=3)
rf_grid_4.fit(X_train_4, y_train_4)

print(rf_grid_4.best_score_)
print(rf_grid_4.best_params_)

-2.3323612763795016
{'max_depth': 10, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 50}


In [ ]:
pred_rf_4 = rf_grid_4.predict(X_val_4)
sqrt(mean_squared_error(y_val_4, pred_rf_4))

6.924242162082397

## 4.4 **GBM**

In [ ]:
# 라이브러리 임포트
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# 1. 모든 칼럼을 포함한 데이터를 활용한 모델링
gbm_reg = GradientBoostingRegressor()
params = {
    'n_estimators' : [100],
    'learning_rate': [0.1, 0.01],
    'max_depth': [4, 8],
    'min_samples_leaf': [100,150],
    'max_features': [0.3, 0.1] 
}

gbm_grid = GridSearchCV(gbm_reg, param_grid = params,scoring='neg_mean_absolute_error', cv=3, verbose=2)
gbm_grid.fit(X_train, y_train)

print(gbm_grid.best_score_)
print(gbm_grid.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   0.4s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   

In [ ]:
pred_gbm = gbm_grid.predict(X_val)
sqrt(mean_squared_error(y_val, pred_gbm))

6.857918639221542

In [ ]:
# 2. 기타 칼럼 제거한 데이터를 활용한 모델링
gbm_reg = GradientBoostingRegressor()
params = {
    'n_estimators' : [100],
    'learning_rate': [0.1, 0.01],
    'max_depth': [4, 8],
    'min_samples_leaf': [100,150],
    'max_features': [0.3, 0.1] 
}

gbm_grid_2 = GridSearchCV(gbm_reg, param_grid = params, cv=3, verbose=2)
gbm_grid_2.fit(X_train_2, y_train_2)

print(gbm_grid_2.best_score_)
print(gbm_grid_2.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   1.0s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   0.4s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   

In [ ]:
pred_gbm_2 = gbm_grid_2.predict(X_val_2)
sqrt(mean_squared_error(y_val_2, pred_gbm_2))

6.784107438839839

In [ ]:
# 3. 동, 기타 칼럼 제거한 데이터를 활용한 모델링
gbm_reg = GradientBoostingRegressor()
params = {
    'n_estimators' : [100],
    'learning_rate': [0.1, 0.01],
    'max_depth': [4, 8],
    'min_samples_leaf': [100,150],
    'max_features': [0.3, 0.1] 
}

gbm_grid_3 = GridSearchCV(gbm_reg, param_grid = params, cv=3, verbose=2)
gbm_grid_3.fit(X_train_3, y_train_3)

print(gbm_grid_3.best_score_)
print(gbm_grid_3.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.5s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.4s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   

In [ ]:
pred_gbm_3 = gbm_grid_3.predict(X_val_3)
sqrt(mean_squared_error(y_val_3, pred_gbm_3))

6.808464703929671

In [ ]:
# 4. 동, 시군구, 기타 칼럼 제거한 데이터를 활용한 모델링
gbm_reg = GradientBoostingRegressor()
params = {
    'n_estimators' : [100],
    'learning_rate': [0.1, 0.01],
    'max_depth': [4, 8],
    'min_samples_leaf': [100,150],
    'max_features': [0.3, 0.1] 
}

gbm_grid_4 = GridSearchCV(gbm_reg, param_grid = params, cv=3, verbose=2)
gbm_grid_4.fit(X_train_4, y_train_4)

print(gbm_grid_4.best_score_)
print(gbm_grid_4.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.2s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.2s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=100, n_estimators=100; total time=   0.2s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.3s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.2s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.3, min_samples_leaf=150, n_estimators=100; total time=   0.2s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   0.2s
[CV] END learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=100, n_estimators=100; total time=   

In [ ]:
pred_gbm_4 = gbm_grid_4.predict(X_val_4)
sqrt(mean_squared_error(y_val_4, pred_gbm_4))

6.956142847875428

## **4.5 XGBoost**

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt

In [ ]:
xgb_reg = XGBRegressor()

params = {
    'n_estimators': [100,200,300,400,500],
    'max_depth': [3,4,5,6,7],
    'eta': [0.01,0.05,0.1,0.5,1.0] ,
    'eval_set': [(X_train,y_train),(X_val,y_val)],
}

xgb_reg = GridSearchCV(xgb_reg, param_grid = params, cv=3, verbose=2)
xgb_reg.fit(X_train, y_train)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        ..
29802    8
5390     3
860      6
15795    4
23654    3
Name: 운송장_건수, Length: 25347, dtype: int64), max_depth=4, n_estimators=300; total time=   2.8s
[00:55:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV] END eta=1.0, eval_set=(        물품_카테고리  송하인_시도  송하인_시군구     송하인_동    송하인_기타  수하인_시도  수하인_시군구  \
17151  0.272727   1.000    0.996  0.257018  0.292044  0.8750    0.832   
7759   0.272727   1.000    0.996  0.294737  0.896274  0.1250    0.168   
18131  0.272727   1.000    0.996  0.224561  0.677744  0.5000    0.364   
20739  0.333333   0.875    0.812  0.178070  0.654582  1.0000    1.000   
23151  0.272727   1.000    0.996  0.224561  0.677744  0.0000    0.040   
...         ...     ...      ...       ...       ...     ...      ...   
29802  0.272727   1.000    1.000  0.884211  0.959718  0.5000    0.468   
5390   0.272727   1.000    1.000  0.892982  0.624371  0.8750    0

In [ ]:
print(xgb_reg.best_score_)
print(xgb_reg.best_params_)

0.1289409112552914
{'eta': 0.01, 'eval_set': (        물품_카테고리  송하인_시도  송하인_시군구     송하인_동    송하인_기타  수하인_시도  수하인_시군구  \
17151  0.272727   1.000    0.996  0.257018  0.292044  0.8750    0.832   
7759   0.272727   1.000    0.996  0.294737  0.896274  0.1250    0.168   
18131  0.272727   1.000    0.996  0.224561  0.677744  0.5000    0.364   
20739  0.333333   0.875    0.812  0.178070  0.654582  1.0000    1.000   
23151  0.272727   1.000    0.996  0.224561  0.677744  0.0000    0.040   
...         ...     ...      ...       ...       ...     ...      ...   
29802  0.272727   1.000    1.000  0.884211  0.959718  0.5000    0.468   
5390   0.272727   1.000    1.000  0.892982  0.624371  0.8750    0.816   
860    0.272727   1.000    1.000  0.866667  0.213494  0.5000    0.472   
15795  0.272727   1.000    0.996  0.294737  0.896274  0.3125    0.264   
23654  0.272727   1.000    0.996  0.257018  0.292044  0.0000    0.052   

          수하인_동    수하인_기타  
17151  0.182296  0.968781  
7759   0.036810  0.54

In [ ]:
# 학습된 모델로 예측 데이터 생성
best_xgb = xgb_reg.best_estimator_
best_xgb

XGBRegressor(eta=0.01,
             eval_set=(        물품_카테고리  송하인_시도  송하인_시군구     송하인_동    송하인_기타  수하인_시도  수하인_시군구  \
17151  0.272727   1.000    0.996  0.257018  0.292044  0.8750    0.832   
7759   0.272727   1.000    0.996  0.294737  0.896274  0.1250    0.168   
18131  0.272727   1.000    0.996  0.224561  0.677744  0.5000    0.364   
20739  0.333333   0.875    0.812  0.178070  0.654582  1.0000    1.000   
23151  0.272727   1.000    0.996  0.224561  0.677744  0.0000    0.040   
...         ...     ...      ...       ...       ...     ...      ......
23654  0.272727   1.000    0.996  0.257018  0.292044  0.0000    0.052   

          수하인_동    수하인_기타  
17151  0.182296  0.968781  
7759   0.036810  0.545821  
18131  0.079755  0.565962  
20739  0.924628  0.846928  
23151  0.008764  0.977845  
...         ...       ...  
29802  0.102542  0.020141  
5390   0.178791  0.171198  
860    0.103418  0.576032  
15795  0.057844  0.152064  
23654  0.011394  0.154079  

[25347 rows x 9 columns],
      

In [ ]:
from sklearn.metrics import mean_squared_error
xgb_pred = xgb_reg.predict(X_val)
sqrt(mean_squared_error(y_val, xgb_pred))

6.226472549819068

## 4.6 **SVM**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt

In [ ]:
svm_clf = SVC(random_state=42)
params = {
    'kernel' : ['poly', 'rbf', 'sigmoid'],
    'gamma' : [0.01,0.05,0.1,0.5,1]
}
svm_clf = GridSearchCV(svm_clf, param_grid = params, cv=3, verbose=0)
svm_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


GridSearchCV(cv=3, estimator=SVC(random_state=42),
             param_grid={'gamma': [0.01, 0.05, 0.1, 0.5, 1],
                         'kernel': ['poly', 'rbf', 'sigmoid']})

In [ ]:
print(svm_clf.best_score_)
print(svm_clf.best_params_)

0.5560815875646034
{'gamma': 0.01, 'kernel': 'poly'}


In [ ]:
from sklearn.metrics import mean_squared_error
svm_pred = svm_clf.predict(X_val)
sqrt(mean_squared_error(y_val, svm_pred))

6.463441576130164

## **4.7 LightGBM**

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV

lgbm_params = {'n_estimators': [350],
               'max_depth': [3],
               'learning_rate': [0.03],
               'num_leaves': [7],
               'min_data_in_leaf':[7]
               }

In [ ]:
lgbm_grid = GridSearchCV(LGBMRegressor(), param_grid=lgbm_params, cv=3, verbose=0, n_jobs=1, scoring='neg_mean_absolute_error')

In [ ]:
lgbm_grid.fit(X_train_new, y_train_new)

GridSearchCV(cv=3, estimator=LGBMRegressor(), n_jobs=1,
             param_grid={'learning_rate': [0.02, 0.03], 'max_depth': [3],
                         'min_data_in_leaf': [7, 8], 'n_estimators': [350],
                         'num_leaves': [7]},
             scoring='neg_mean_absolute_error')

In [ ]:
lgbm_grid.best_params_

{'learning_rate': 0.03,
 'max_depth': 3,
 'min_data_in_leaf': 7,
 'n_estimators': 350,
 'num_leaves': 7}

In [ ]:
lgbm_grid.best_score_

-2.401222392241058

In [ ]:
lgbm = LGBMRegressor(learning_rate=0.03, max_depth=3, n_estimators=350, min_data_in_leaf=8, num_leaves=8)
lgbm.fit(X_train_new, y_train_new)
pred_lgbm = lgbm.predict(X_test)
mean_squared_error(y_test, pred_lgbm) ** 0.5

5.4989392900537695

## **4.8 ExtraTrees**

In [ ]:
from sklearn.model_selection import GridSearchCV

extc_params = {'max_depth': [None],
               'max_features': [5],
               'min_samples_split': [2],
               'min_samples_leaf': [5],
               'bootstrap': [False],
               'n_estimators': [300]}

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
extc_grid = GridSearchCV(ExtraTreesRegressor(random_state=42), param_grid=extc_params, cv=3, verbose=0, n_jobs=1, scoring='neg_mean_absolute_error')

In [ ]:
extc_grid.fit(X_train_new, y_train_new)

GridSearchCV(cv=3, estimator=ExtraTreesRegressor(random_state=42), n_jobs=1,
             param_grid={'bootstrap': [False], 'max_depth': [None],
                         'max_features': [5], 'min_samples_leaf': [3, 5, 7],
                         'min_samples_split': [2],
                         'n_estimators': [250, 300, 350]},
             scoring='neg_mean_absolute_error')

In [ ]:
extc_grid.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 5,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 300}

In [ ]:
extc_grid.best_score_

-2.3889234951466243

In [ ]:
extc = ExtraTreesRegressor(random_state=42, bootstrap=False, max_depth=None, max_features=5, min_samples_leaf=5, min_samples_split=2, n_estimators=300)
extc.fit(X_train_new, y_train_new)
pred_extc = extc.predict(X_test)
mean_squared_error(y_test, pred_extc) ** 0.5

5.418900615006726

## **4.9 Voting Regressor**

In [ ]:
from sklearn.ensemble import VotingRegressor
voting_reg = VotingRegressor(
    estimators=[('lgbm_reg',lgbm), 
                ('extc_reg',extc)])
voting_reg.fit(X_train_new, y_train_new)

VotingRegressor(estimators=[('lgbm_reg',
                             LGBMRegressor(learning_rate=0.03, max_depth=3,
                                           min_data_in_leaf=8, n_estimators=350,
                                           num_leaves=8)),
                            ('extc_reg',
                             ExtraTreesRegressor(max_features=5,
                                                 min_samples_leaf=5,
                                                 n_estimators=300,
                                                 random_state=42))])

In [ ]:
pred_voting = voting_reg.predict(X_test)
mean_squared_error(y_test, pred_voting) ** 0.5

5.387792623465095

## **4.10 Catboost**

In [ ]:
# 라이브러리 임포트
import numpy as np
from sklearn.model_selection import train_test_split
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
!pip install optuna
import optuna
from optuna.samplers import TPESampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 19 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 65.6 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 112 kB 59.6 MB/s 
     |████████████████████████████████| 147 kB 69.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=84a92fdaac74d059a9d15b4c7a92c90accba18812d86f0f806b901dfa7df65c9
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
# 하이퍼파라미터 튜닝을 위한 함수
def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }
  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  cat_features = range(test.shape[1])
  cat = CatBoostRegressor(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict(X_valid)
  rmse = np.sqrt(mean_squared_error(y_valid, cat_pred))

  return rmse

In [ ]:
# 최적의 파라미터 선정
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)

study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-10-07 03:55:35,210] A new study created in memory with name: cat_parameter_opt
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  


0:	learn: 6.9849564	test: 6.9815548	test1: 5.7121936	best: 5.7121936 (0)	total: 200ms	remaining: 12m 38s
100:	learn: 6.5984842	test: 6.4980547	test1: 5.5779865	best: 5.5779865 (100)	total: 11.3s	remaining: 6m 53s
200:	learn: 6.4315274	test: 6.3390073	test1: 5.5574746	best: 5.5574746 (200)	total: 21.8s	remaining: 6m 30s
300:	learn: 6.3149988	test: 6.2527369	test1: 5.5492010	best: 5.5479438 (293)	total: 32.3s	remaining: 6m 14s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.547943804
bestIteration = 293

Shrink model to first 294 iterations.


[I 2022-10-07 03:56:10,537] Trial 0 finished with value: 5.547943803792508 and parameters: {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
 

0:	learn: 6.8814640	test: 6.8808386	test1: 6.2000941	best: 6.2000941 (0)	total: 47.8ms	remaining: 1m 9s
100:	learn: 6.5571220	test: 6.5471577	test1: 6.0594492	best: 6.0594492 (100)	total: 4.2s	remaining: 56.3s
200:	learn: 6.4678670	test: 6.4870391	test1: 6.0437005	best: 6.0432840 (197)	total: 8.25s	remaining: 51.5s
300:	learn: 6.1776318	test: 6.2961745	test1: 6.0234963	best: 6.0230260 (299)	total: 12.9s	remaining: 49.6s
400:	learn: 5.8798136	test: 6.1586127	test1: 5.9970118	best: 5.9959323 (370)	total: 18.1s	remaining: 47.5s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.995932269
bestIteration = 370

Shrink model to first 371 iterations.


[I 2022-10-07 03:56:29,268] Trial 1 finished with value: 5.995932268768682 and parameters: {'learning_rate': 0.04763628595029446, 'bagging_temperature': 21.368329072358772, 'n_estimators': 1455, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  


...

0:	learn: 6.8759748	test: 6.8759837	test1: 6.2552583	best: 6.2552583 (0)	total: 19.6ms	remaining: 20.3s
100:	learn: 6.5574785	test: 6.5551872	test1: 6.0834723	best: 6.0834723 (100)	total: 15.5s	remaining: 2m 23s
200:	learn: 6.3859669	test: 6.4714473	test1: 6.0503963	best: 6.0497625 (193)	total: 34.1s	remaining: 2m 22s
300:	learn: 6.0854002	test: 6.4031883	test1: 6.0045020	best: 6.0025812 (282)	total: 55.1s	remaining: 2m 14s
400:	learn: 5.1249428	test: 6.2556097	test1: 5.9208375	best: 5.9149952 (369)	total: 1m 34s	remaining: 2m 30s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.914995189
bestIteration = 369

Shrink model to first 370 iterations.


[I 2022-10-07 04:15:31,687] Trial 8 finished with value: 5.914995189352973 and parameters: {'learning_rate': 0.04169990777997927, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 1038, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  


0:	learn: 6.6572355	test: 6.6566506	test1: 7.1287336	best: 7.1287336 (0)	total: 148ms	remaining: 8m 18s
100:	learn: 6.5516459	test: 6.5159375	test1: 7.0282229	best: 7.0282229 (100)	total: 6.25s	remaining: 3m 21s
200:	learn: 6.5042539	test: 6.4526781	test1: 6.9935713	best: 6.9935713 (200)	total: 12.1s	remaining: 3m 11s
300:	learn: 6.4569394	test: 6.3964352	test1: 6.9655505	best: 6.9655505 (300)	total: 18.3s	remaining: 3m 6s
400:	learn: 6.4169925	test: 6.3406991	test1: 6.9438259	best: 6.9438259 (400)	total: 24.8s	remaining: 3m 3s
500:	learn: 6.3842751	test: 6.3084610	test1: 6.9271256	best: 6.9271255 (499)	total: 31.2s	remaining: 2m 58s
600:	learn: 6.3532486	test: 6.2718788	test1: 6.9093296	best: 6.9093224 (599)	total: 37.9s	remaining: 2m 54s
700:	learn: 6.3369931	test: 6.2535562	test1: 6.9030123	best: 6.9030123 (700)	total: 45s	remaining: 2m 51s
800:	learn: 6.3007271	test: 6.2314240	test1: 6.8920949	best: 6.8920824 (794)	total: 51.5s	remaining: 2m 44s
900:	learn: 6.2584264	test: 6.204557

[I 2022-10-07 04:17:34,907] Trial 9 finished with value: 6.689094036879517 and parameters: {'learning_rate': 0.011896326393433528, 'bagging_temperature': 0.0133572404119741, 'n_estimators': 3364, 'max_depth': 8, 'random_strength': 51, 'colsample_bylevel': 0.9445398843556558, 'l2_leaf_reg': 7.486273952174759e-06, 'min_child_samples': 44, 'max_bin': 427, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.


Best Score: 5.547943803792508
Best trial {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}


In [ ]:
# 최적 파라미터를 활용한 Catboost 모델 적합
cat_reg = CatBoostRegressor(
              learning_rate = 0.011265617213006592, 
              bagging_temperature = 0.27155819552829413, 
              n_estimators = 1021, 
              max_depth = 15, 
              random_strength = 62, 
              colsample_bylevel = 0.5985388149115896, 
              l2_leaf_reg = 1.9161149250778487e-06, 
              min_child_samples = 34, 
              max_bin = 297, 
              od_type = 'IncToDec')
cat_reg.fit(X, y)

0:	learn: 6.7548897	total: 780ms	remaining: 13m 15s
1:	learn: 6.7526894	total: 1.5s	remaining: 12m 44s
2:	learn: 6.7465505	total: 2.42s	remaining: 13m 39s
3:	learn: 6.7336148	total: 3.22s	remaining: 13m 39s
4:	learn: 6.7326541	total: 3.89s	remaining: 13m 10s
5:	learn: 6.7265321	total: 4.58s	remaining: 12m 55s
6:	learn: 6.7241438	total: 5.26s	remaining: 12m 42s
7:	learn: 6.7233091	total: 5.27s	remaining: 11m 7s
8:	learn: 6.7217024	total: 5.89s	remaining: 11m 2s
9:	learn: 6.7206717	total: 6.3s	remaining: 10m 37s
10:	learn: 6.7187514	total: 7.07s	remaining: 10m 48s
11:	learn: 6.7167385	total: 7.78s	remaining: 10m 54s
12:	learn: 6.7151699	total: 8.29s	remaining: 10m 43s
13:	learn: 6.7142351	total: 8.91s	remaining: 10m 40s
14:	learn: 6.7075506	total: 9.43s	remaining: 10m 32s
15:	learn: 6.7056384	total: 10.1s	remaining: 10m 35s
16:	learn: 6.6996284	total: 10.8s	remaining: 10m 39s
17:	learn: 6.6978370	total: 11.5s	remaining: 10m 42s
18:	learn: 6.6965764	total: 12.1s	remaining: 10m 37s
19:	lea

In [ ]:
# 예측
cat_pred = cat_reg.predict(test)

In [ ]:
# Catboost 예측값 확인
cat_pred

array([4.75122581, 4.68496398, 4.94029887, ..., 3.74214544, 3.89348139,
       4.43611772])

In [ ]:
# 예측값을 submission 파일에 저장
cat_pred = pd.DataFrame(cat_pred)
submission['운송장_건수'] = cat_pred
submission

,index,운송장_건수
0,0,4.751226
1,1,4.684964
2,2,4.940299
3,3,4.965951
4,4,4.959160
...,...,...
7915,7915,4.954910
7916,7916,5.211716
7917,7917,3.742145
7918,7918,3.893481


## **4.11 KNN**

In [ ]:
# 라이브러리 임포트
import numpy as np
from sklearn.model_selection import train_test_split
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
!pip install optuna
import optuna
from optuna.samplers import TPESampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 19 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 65.6 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 112 kB 59.6 MB/s 
     |████████████████████████████████| 147 kB 69.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=84a92fdaac74d059a9d15b4c7a92c90accba18812d86f0f806b901dfa7df65c9
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
# 하이퍼파라미터 튜닝을 위한 함수
def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }
  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  cat_features = range(test.shape[1])
  cat = CatBoostRegressor(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict(X_valid)
  rmse = np.sqrt(mean_squared_error(y_valid, cat_pred))

  return rmse

In [ ]:
# 최적의 파라미터 선정
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)

study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-10-07 03:55:35,210] A new study created in memory with name: cat_parameter_opt
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  


0:	learn: 6.9849564	test: 6.9815548	test1: 5.7121936	best: 5.7121936 (0)	total: 200ms	remaining: 12m 38s
100:	learn: 6.5984842	test: 6.4980547	test1: 5.5779865	best: 5.5779865 (100)	total: 11.3s	remaining: 6m 53s
200:	learn: 6.4315274	test: 6.3390073	test1: 5.5574746	best: 5.5574746 (200)	total: 21.8s	remaining: 6m 30s
300:	learn: 6.3149988	test: 6.2527369	test1: 5.5492010	best: 5.5479438 (293)	total: 32.3s	remaining: 6m 14s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.547943804
bestIteration = 293

Shrink model to first 294 iterations.


[I 2022-10-07 03:56:10,537] Trial 0 finished with value: 5.547943803792508 and parameters: {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
 

0:	learn: 6.8814640	test: 6.8808386	test1: 6.2000941	best: 6.2000941 (0)	total: 47.8ms	remaining: 1m 9s
100:	learn: 6.5571220	test: 6.5471577	test1: 6.0594492	best: 6.0594492 (100)	total: 4.2s	remaining: 56.3s
200:	learn: 6.4678670	test: 6.4870391	test1: 6.0437005	best: 6.0432840 (197)	total: 8.25s	remaining: 51.5s
300:	learn: 6.1776318	test: 6.2961745	test1: 6.0234963	best: 6.0230260 (299)	total: 12.9s	remaining: 49.6s
400:	learn: 5.8798136	test: 6.1586127	test1: 5.9970118	best: 5.9959323 (370)	total: 18.1s	remaining: 47.5s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.995932269
bestIteration = 370

Shrink model to first 371 iterations.


[I 2022-10-07 03:56:29,268] Trial 1 finished with value: 5.995932268768682 and parameters: {'learning_rate': 0.04763628595029446, 'bagging_temperature': 21.368329072358772, 'n_estimators': 1455, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  


...

0:	learn: 6.8759748	test: 6.8759837	test1: 6.2552583	best: 6.2552583 (0)	total: 19.6ms	remaining: 20.3s
100:	learn: 6.5574785	test: 6.5551872	test1: 6.0834723	best: 6.0834723 (100)	total: 15.5s	remaining: 2m 23s
200:	learn: 6.3859669	test: 6.4714473	test1: 6.0503963	best: 6.0497625 (193)	total: 34.1s	remaining: 2m 22s
300:	learn: 6.0854002	test: 6.4031883	test1: 6.0045020	best: 6.0025812 (282)	total: 55.1s	remaining: 2m 14s
400:	learn: 5.1249428	test: 6.2556097	test1: 5.9208375	best: 5.9149952 (369)	total: 1m 34s	remaining: 2m 30s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 5.914995189
bestIteration = 369

Shrink model to first 370 iterations.


[I 2022-10-07 04:15:31,687] Trial 8 finished with value: 5.914995189352973 and parameters: {'learning_rate': 0.04169990777997927, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 1038, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  


0:	learn: 6.6572355	test: 6.6566506	test1: 7.1287336	best: 7.1287336 (0)	total: 148ms	remaining: 8m 18s
100:	learn: 6.5516459	test: 6.5159375	test1: 7.0282229	best: 7.0282229 (100)	total: 6.25s	remaining: 3m 21s
200:	learn: 6.5042539	test: 6.4526781	test1: 6.9935713	best: 6.9935713 (200)	total: 12.1s	remaining: 3m 11s
300:	learn: 6.4569394	test: 6.3964352	test1: 6.9655505	best: 6.9655505 (300)	total: 18.3s	remaining: 3m 6s
400:	learn: 6.4169925	test: 6.3406991	test1: 6.9438259	best: 6.9438259 (400)	total: 24.8s	remaining: 3m 3s
500:	learn: 6.3842751	test: 6.3084610	test1: 6.9271256	best: 6.9271255 (499)	total: 31.2s	remaining: 2m 58s
600:	learn: 6.3532486	test: 6.2718788	test1: 6.9093296	best: 6.9093224 (599)	total: 37.9s	remaining: 2m 54s
700:	learn: 6.3369931	test: 6.2535562	test1: 6.9030123	best: 6.9030123 (700)	total: 45s	remaining: 2m 51s
800:	learn: 6.3007271	test: 6.2314240	test1: 6.8920949	best: 6.8920824 (794)	total: 51.5s	remaining: 2m 44s
900:	learn: 6.2584264	test: 6.204557

[I 2022-10-07 04:17:34,907] Trial 9 finished with value: 6.689094036879517 and parameters: {'learning_rate': 0.011896326393433528, 'bagging_temperature': 0.0133572404119741, 'n_estimators': 3364, 'max_depth': 8, 'random_strength': 51, 'colsample_bylevel': 0.9445398843556558, 'l2_leaf_reg': 7.486273952174759e-06, 'min_child_samples': 44, 'max_bin': 427, 'od_type': 'IncToDec'}. Best is trial 0 with value: 5.547943803792508.


Best Score: 5.547943803792508
Best trial {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}


In [ ]:
# 최적 파라미터를 활용한 Catboost 모델 적합
cat_reg = CatBoostRegressor(
              learning_rate = 0.011265617213006592, 
              bagging_temperature = 0.27155819552829413, 
              n_estimators = 1021, 
              max_depth = 15, 
              random_strength = 62, 
              colsample_bylevel = 0.5985388149115896, 
              l2_leaf_reg = 1.9161149250778487e-06, 
              min_child_samples = 34, 
              max_bin = 297, 
              od_type = 'IncToDec')
cat_reg.fit(X, y)

0:	learn: 6.7548897	total: 780ms	remaining: 13m 15s
1:	learn: 6.7526894	total: 1.5s	remaining: 12m 44s
2:	learn: 6.7465505	total: 2.42s	remaining: 13m 39s
3:	learn: 6.7336148	total: 3.22s	remaining: 13m 39s
4:	learn: 6.7326541	total: 3.89s	remaining: 13m 10s
5:	learn: 6.7265321	total: 4.58s	remaining: 12m 55s
6:	learn: 6.7241438	total: 5.26s	remaining: 12m 42s
7:	learn: 6.7233091	total: 5.27s	remaining: 11m 7s
8:	learn: 6.7217024	total: 5.89s	remaining: 11m 2s
9:	learn: 6.7206717	total: 6.3s	remaining: 10m 37s
10:	learn: 6.7187514	total: 7.07s	remaining: 10m 48s
11:	learn: 6.7167385	total: 7.78s	remaining: 10m 54s
12:	learn: 6.7151699	total: 8.29s	remaining: 10m 43s
13:	learn: 6.7142351	total: 8.91s	remaining: 10m 40s
14:	learn: 6.7075506	total: 9.43s	remaining: 10m 32s
15:	learn: 6.7056384	total: 10.1s	remaining: 10m 35s
16:	learn: 6.6996284	total: 10.8s	remaining: 10m 39s
17:	learn: 6.6978370	total: 11.5s	remaining: 10m 42s
18:	learn: 6.6965764	total: 12.1s	remaining: 10m 37s
19:	lea

In [ ]:
# 예측
cat_pred = cat_reg.predict(test)

In [ ]:
# Catboost 예측값 확인
cat_pred

array([4.75122581, 4.68496398, 4.94029887, ..., 3.74214544, 3.89348139,
       4.43611772])

In [ ]:
# 예측값을 submission 파일에 저장
cat_pred = pd.DataFrame(cat_pred)
submission['운송장_건수'] = cat_pred
submission

,index,운송장_건수
0,0,4.751226
1,1,4.684964
2,2,4.940299
3,3,4.965951
4,4,4.959160
...,...,...
7915,7915,4.954910
7916,7916,5.211716
7917,7917,3.742145
7918,7918,3.893481


---

# **5. 제출파일 생성**

  - RMSE값이 가장 작은 XGBoost 모델링을 최종 모델로 결정하였다.

In [ ]:
X_train = train.drop(['운송장_건수','송하인_기타','수하인_기타'], axis=1)
y_train = train['운송장_건수']
test = test.drop(['송하인_기타','수하인_기타'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, test_size = 0.3,random_state = 42)

In [ ]:
params = {
    'n_estimators': [80],
    'max_depth': [4],
    'eta': [0.01] ,
    'eval_set': [(X_train_new,y_train_new),(X_val,y_val)],
}

xgb_reg = GridSearchCV(XGBRegressor(random_state=42), param_grid = params, cv=3, verbose=0, scoring='neg_mean_squared_error')
xgb_reg.fit(X_train_new, y_train_new)
xgb_pred = xgb_reg.predict(X_val)
sqrt(mean_squared_error(xgb_pred, y_val))

[05:28:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:28:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:28:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:28:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:28:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:28:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:28:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


6.033750800857901

In [ ]:
xgb_reg = XGGRegressor(n_estimators = 80, max_depth = 4, eta = 0.01)
xgb_reg.fit(X_train, y_train)
pred_final = xgb_reg.predict(test)

In [ ]:
# 최종 제출 파일
submission['운송장_건수'] = pred_final
submission.to_csv('submission_xgb_reg.csv',index=False)